# 1. 코사인 유사도(Cosine Similarity)
- BoW, DTM, TF-IDF, Word2Vec로 단어를 수치화한 다음 문서의 유사도를 구해야 하는데 그 방법 중 하나가 **코사인 유사도**

## 1) 코사인 유사도(Cosine Similarity)
For two vectors $\overrightarrow{A}, \overrightarrow{B}$,

\begin{align}
similarity = cos(\theta) = \frac{A \cdot B}{||A|| ||B||} = \frac{\sum_{i=1}^{n}A_i \times B_i}{\sqrt{\sum_{i=1}^{n}(A_i)^2} \times \sqrt{\sum_{i=1}^{n}(B_i)^2}}
\end{align}

문서 단어 행렬이나 TF-IDF 행렬을 통해서 문서의 유사도를 구하는 경우에는 <ins>문서 단어 행렬</ins>이나 <ins>TF-IDF 행렬</ins>이 각각의 특징 벡터 A, B가 됌


- Example
    - 문서1 : 저는 사과 좋아요
    - 문서2 : 저는 바나나 좋아요
    - 문서3 : 저는 바나나 좋아요 저는 바나나 좋아요

|-|바나나|사과|저는|좋아요|
|---|---|---|---|---|
|문서1|0|1|1|1|
|문서2|1|0|1|1|
|문서3|2|0|2|2|

In [1]:
from numpy import dot
from numpy.linalg import norm
import numpy as np
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))
    
doc1=np.array([0,1,1,1])
doc2=np.array([1,0,1,1])
doc3=np.array([2,0,2,2])

print(cos_sim(doc1, doc2)) #문서1과 문서2의 코사인 유사도
print(cos_sim(doc1, doc3)) #문서1과 문서3의 코사인 유사도
print(cos_sim(doc2, doc3)) #문서2과 문서3의 코사인 유사도

0.6666666666666667
0.6666666666666667
1.0000000000000002


- 코사인 유사도는 문서의 길이가 다른 상황에서 비교적 공정한 비교를 할 수 있도록 도와준다.
    - 벡터의 크기가 아니라 벡터의 방향(패턴)에 초점을 두기 때문

## 2) 유사도를 이용한 추천 시스템 구현하기

### 캐글에서 사용되었던 영화 데이터셋을 가지고 영화 추천 시스템 만들기

- [다운로드](https://www.kaggle.com/rounakbanik/the-movies-dataset)
    - 여기서 **movies_metadata.csv** 다운로드 받기

In [2]:
import pandas as pd
data = pd.read_csv('./movies_metadata.csv', low_memory=False)
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [3]:
# 20000개의 샘플만 가지고 해보기
data=data.head(20000)


print(data['overview'].isnull().sum() ) # overview 컬럼에서 총 135개의 Null 값이 있음
data['overview'] = data['overview'].fillna('') # Null 대신 empty value 넣기
print(data['overview'].isnull().sum() )

135
0


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
# overview에 대해서 tf-idf 수행
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['overview'])

# 20000개의 영화를 표현하기 위해 총 47487개의 단어 사용
print(tfidf_matrix.shape) 

(20000, 47487)


In [6]:
# 이거 50초 정도 걸림...
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
print(cosine_sim)

[[1.         0.01575748 0.         ... 0.         0.         0.        ]
 [0.01575748 1.         0.04907345 ... 0.         0.         0.        ]
 [0.         0.04907345 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.08375766]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.08375766 0.         1.        ]]


In [7]:
# 영화의 타이틀과 인덱스를 가진 테이블
# 이 테이블의 용도는 영화의 타이틀을 입력하면 인덱스 리턴 받으려고
indices = pd.Series(data.index, index=data['title']).drop_duplicates()
print(indices.head())

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64


In [8]:
idx = indices['Father of the Bride Part II']
print(idx)

4


In [9]:
# 코사인 유사도를 이용해서 선택한 영화와 overview가 가장 유사한 10개의 영화 찾는 함수

def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 영화를 가지고 연산할 수 있습니다.
    idx = indices[title]

    # 모든 영화에 대해서 해당 영화와의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아옵니다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 받아옵니다.
    movie_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴합니다.
    return data['title'].iloc[movie_indices]

In [10]:
get_recommendations('The Dark Knight Rises')

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object